## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [7]:
type(x_train)

numpy.ndarray

In [11]:
print(mean_train)
type(mean_train)

120.70756512369792


numpy.float64

In [12]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [17]:
# build a classfier
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters = 32, kernel_size = (3,3),input_shape=(32,32,3),activation='relu'))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(filters = 64, kernel_size = (5,5),input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='relu'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=10)`


Epoch 1/10
50000/50000 [==============================] - 247s 5ms/step - loss: 2.0868 - accuracy: 0.3392
Epoch 2/10
50000/50000 [==============================] - 249s 5ms/step - loss: 2.1055 - accuracy: 0.3201
Epoch 3/10
50000/50000 [==============================] - 237s 5ms/step - loss: 1.8878 - accuracy: 0.3506
Epoch 4/10
50000/50000 [==============================] - 228s 5ms/step - loss: 1.9418 - accuracy: 0.3219
Epoch 5/10
50000/50000 [==============================] - 238s 5ms/step - loss: 1.7601 - accuracy: 0.4158
Epoch 6/10
50000/50000 [==============================] - 243s 5ms/step - loss: 1.7494 - accuracy: 0.4144
Epoch 7/10
50000/50000 [==============================] - 241s 5ms/step - loss: 1.6813 - accuracy: 0.4559
Epoch 8/10
50000/50000 [==============================] - 250s 5ms/step - loss: 1.7391 - accuracy: 0.4213
Epoch 9/10
50000/50000 [==============================] - 244s 5ms/step - loss: 1.6585 - accuracy: 0.4502
Epoch 10/10
50000/50000 [=====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [18]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[116.24664 , 185.69298 , 127.23918 , 411.79068 , 159.66815 ,
        249.06355 , 325.29333 ,  52.473087,  24.071903,   5.031333]],
      dtype=float32)